In [6]:
import pandas as pd
import numpy as np

train = pd.read_csv('../data/train.csv').drop(columns='ID')
test = pd.read_csv('../data/test.csv').drop(columns='ID')
submission = pd.read_csv('../data/sample_submission.csv')

In [20]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.X = data.drop(columns='y').values
        self.y = data['y'].values

    def __len__(self):
        return len(self.data)        
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.float32)
    
train_dataset = CustomDataset(train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [13]:
class DiffusionModel(nn.Module):
    def __init__(self, input_dim, latent_dim=64):
        super(DiffusionModel, self).__init__()
        self.input_dim = input_dim
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = DiffusionModel(input_dim=train.shape[1]-1)

In [14]:
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

In [15]:
from tqdm import tqdm

def loss_reweighting(pred, target, weights):
    return (weights*(pred-target)**2).mean()

def forward_diffusion(x, noise_std):
    noise = torch.randn_like(x) ** noise_std
    return x+noise

def reverse_diffusion(xt, score_fn):
    return xt - score_fn(xt)


# train
for epoch in range(100):
    model.train()
    
    for X, y in tqdm(train_loader):
        optimizer.zero_grad()
        noise_std = 0.1
        xt = forward_diffusion(X, noise_std)

        # loss reweighting
        pred = model(xt)
        weights = 1 / (1+y.abs()) 
        loss = loss_reweighting(pred, y, weights)
        
        loss.backward()
        optimizer.step() 

100%|██████████| 1254/1254 [00:01<00:00, 1127.33it/s]


In [18]:
# prediction
model.eval()
test_X = torch.tensor(test.values, dtype=torch.float32)
predictions = model(test_X).detach().numpy()

In [19]:
# results
submission['y'] = predictions
submission.to_csv('submission.csv', index =False)